In [3]:
import sqlite3
import pandas as pd
import numpy as np

In [4]:
# Loading Data
db = sqlite3.connect('../data/database.sqlite')
#country = pd.read_sql_query('SELECT * FROM Country', db)
#league = pd.read_sql_query('SELECT * FROM League', db)
match = pd.read_sql_query('SELECT * FROM Match', db)
player = pd.read_sql_query('SELECT * FROM Player', db)
player_attributes = pd.read_sql_query('SELECT * FROM Player_Attributes', db)
player_attributes['date'] = pd.to_datetime(player_attributes.date)
team = pd.read_sql_query('SELECT * FROM Team', db)
#team_attributes = pd.read_sql_query('SELECT * FROM Team_Attributes', db)
'''league_season_winners= pd.read_sql_query("""
select league_id, season,team_api_id, team_long_name, max(wins) as wins
from
	(SELECT league_id, season,team_api_id, team_long_name, count(*) as wins
	FROM Match, Team
	WHERE (home_team_api_id==team_api_id AND home_team_goal > away_team_goal) OR (away_team_api_id == team_api_id AND home_team_goal < away_team_goal)
	GROUP By league_id, season,team_api_id, team_long_name)
where wins>10
GROUP by league_id, season;
 """, db)'''
db.close()

In [3]:
league_season_winners.head(18)

,league_id,season,team_api_id,team_long_name,wins
0,1,2008/2009,8635,RSC Anderlecht,24
1,1,2009/2010,8635,RSC Anderlecht,22
2,1,2010/2011,8635,RSC Anderlecht,19
3,1,2011/2012,8635,RSC Anderlecht,20
4,1,2012/2013,8635,RSC Anderlecht,20
5,1,2014/2015,8342,Club Brugge KV,17
6,1,2015/2016,8342,Club Brugge KV,21
7,1729,2008/2009,10260,Manchester United,28
8,1729,2009/2010,8455,Chelsea,27
9,1729,2010/2011,10260,Manchester United,23


In [4]:
winners_md = league_season_winners.sample(5).to_markdown()
winners_md

'|    |   league_id | season    |   team_api_id | team_long_name   |   wins |\n|---:|------------:|:----------|--------------:|:-----------------|-------:|\n|  1 |           1 | 2009/2010 |          8635 | RSC Anderlecht   |     22 |\n| 77 |       21518 | 2014/2015 |          8633 | Real Madrid CF   |     30 |\n| 23 |        7809 | 2008/2009 |          8721 | VfL Wolfsburg    |     21 |\n|  3 |           1 | 2011/2012 |          8635 | RSC Anderlecht   |     20 |\n| 76 |       21518 | 2013/2014 |          9906 | Atlético Madrid  |     28 |'

In [6]:
home_player_cols = [f'home_player_{i}' for i in range(1,12)]
away_player_cols = [f'away_player_{i}' for i in range(1,12)]

match['home_players'] = match[home_player_cols].agg(list, axis=1)
match['away_players'] = match[away_player_cols].agg(list, axis=1)

In [7]:
match = match[['id', 'league_id', 'season', 'stage', 'date', 'match_api_id', 'home_team_api_id', 'away_team_api_id',
               'home_team_goal', 'away_team_goal', 'goal', 'card', 'home_players', 'away_players']].dropna()

match.head(5)

,id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,goal,card,home_players,away_players
1728,1729,1729,2008/2009,1,2008-08-17 00:00:00,489042,10260,10261,1,1,<goal><value><comment>n</comment><stats><goals...,<card><value><comment>y</comment><stats><ycard...,"[30726.0, 30362.0, 30620.0, 30865.0, 32569.0, ...","[24224.0, 25518.0, 24228.0, 30929.0, 29581.0, ..."
1729,1730,1729,2008/2009,1,2008-08-16 00:00:00,489043,9825,8659,1,0,<goal><value><comment>n</comment><stats><goals...,<card />,"[23686.0, 26111.0, 38835.0, 30986.0, 31291.0, ...","[36373.0, 36832.0, 23115.0, 37280.0, 24728.0, ..."
1730,1731,1729,2008/2009,1,2008-08-16 00:00:00,489044,8472,8650,0,1,<goal><value><comment>n</comment><stats><goals...,<card><value><comment>y</comment><stats><ycard...,"[32562.0, 38836.0, 24446.0, 24408.0, 36786.0, ...","[30660.0, 37442.0, 30617.0, 24134.0, 414792.0,..."
1731,1732,1729,2008/2009,1,2008-08-16 00:00:00,489045,8654,8528,2,1,<goal><value><comment>n</comment><stats><goals...,<card><value><comment>y</comment><stats><ycard...,"[36374.0, 30966.0, 23818.0, 37277.0, 30687.0, ...","[34421.0, 34987.0, 35472.0, 111865.0, 25005.0,..."
1732,1733,1729,2008/2009,1,2008-08-17 00:00:00,489046,10252,8456,4,2,<goal><value><comment>n</comment><stats><goals...,<card><value><comment>y</comment><stats><ycard...,"[30380.0, 30357.0, 24658.0, 43280.0, 23282.0, ...","[31432.0, 46403.0, 24208.0, 23939.0, 33963.0, ..."


In [27]:
def unique_players(series):
    players = []
    for entry in series:
        players.extend(entry)

    return list(set(players))

In [28]:
players_per_team_per_season = {}

teams = match.home_team_api_id.append(match.away_team_api_id, ignore_index=True).unique()

for team in teams:
    players_per_team_per_season[team] = {}
    temp = match[match.away_team_api_id == team]
    for season, df in temp.groupby('season'):
        players_per_team_per_season[team][season] = unique_players(df.away_players)

    temp = match[match.away_team_api_id == team]
    for season,df in temp.groupby('season'):
        players_per_team_per_season[team][season].extend(unique_players(df.away_players))
        players_per_team_per_season[team][season] = list(set(players_per_team_per_season[team][season]))

In [29]:
for s in match.away_players:
    print(s)
    break

[24224.0, 25518.0, 24228.0, 30929.0, 29581.0, 38807.0, 40565.0, 30360.0, 33852.0, 34574.0, 37799.0]


In [14]:
for x in match.columns:
    print(x)

id
country_id
league_id
season
stage
date
match_api_id
home_team_api_id
away_team_api_id
home_team_goal
away_team_goal
home_player_X1
home_player_X2
home_player_X3
home_player_X4
home_player_X5
home_player_X6
home_player_X7
home_player_X8
home_player_X9
home_player_X10
home_player_X11
away_player_X1
away_player_X2
away_player_X3
away_player_X4
away_player_X5
away_player_X6
away_player_X7
away_player_X8
away_player_X9
away_player_X10
away_player_X11
home_player_Y1
home_player_Y2
home_player_Y3
home_player_Y4
home_player_Y5
home_player_Y6
home_player_Y7
home_player_Y8
home_player_Y9
home_player_Y10
home_player_Y11
away_player_Y1
away_player_Y2
away_player_Y3
away_player_Y4
away_player_Y5
away_player_Y6
away_player_Y7
away_player_Y8
away_player_Y9
away_player_Y10
away_player_Y11
home_player_1
home_player_2
home_player_3
home_player_4
home_player_5
home_player_6
home_player_7
home_player_8
home_player_9
home_player_10
home_player_11
away_player_1
away_player_2
away_player_3
away_player_4
a

In [51]:
players_per_team_df = pd.DataFrame(players_per_team_per_season).reset_index()

players_per_team_df = players_per_team_df.melt(id_vars='index')
players_per_team_df.columns = ['season', 'team_api_id', 'players']
players_per_team_df.dropna(subset=['players'], inplace=True, axis=0)

players_per_team_df['players'] = players_per_team_df.players.apply(lambda x: [int(y) for y in pd.Series(x).dropna()])

(815, 3)


,season,team_api_id,players
1374,2014/2015,8305,"[75397, 67334, 38427, 209950, 101920, 109222, ..."
865,2009/2010,8543,"[33544, 27657, 30474, 27660, 39444, 33687, 393..."
349,2013/2014,9851,"[473858, 447107, 26374, 159882, 156556, 231199..."
326,2014/2015,10249,"[210692, 202633, 215307, 18576, 192920, 26138,..."
365,2013/2014,9874,"[173955, 26118, 279178, 112012, 26126, 185618,..."


In [ ]:
print(players_per_team_df.shape)
players_per_team_df.sample(5)

In [53]:
players_per_team_df.to_csv('players_per_team.csv', index_label='idx')

In [8]:
players_per_team_df = pd.read_csv('players_per_team.csv', index_col='idx')
players_per_team_df['players'] = players_per_team_df.players.apply(eval)
players_per_team_df.sample(5)

,season,team_api_id,players
idx,,,
884,2012/2013,9857,"[198400, 115330, 41475, 27658, 199833, 39450, ..."
1288,2008/2009,8696,"[37512, 33828, 38567, 35638, 37579, 37581, 338..."
1086,2014/2015,7788,"[179201, 189957, 189958, 45831, 148487, 179206..."
69,2013/2014,8667,"[163200, 42247, 35480, 23839, 24227, 24105, 23..."
461,2013/2014,9837,"[14593, 184591, 26262, 131991, 26264, 26402, 2..."


In [24]:
def calculate_total_score(row):
    l = row.season.split('/')
    start_year, end_year = l[0], l[1]
    start_date = pd.to_datetime(f'01-08-{start_year}')
    end_date = pd.to_datetime(f'01-06-{end_year}')
    temp = player_attributes[np.logical_and(np.logical_and(player_attributes['player_api_id'].isin(row.players), player_attributes.date <= end_date), player_attributes.date >= start_date)][['player_api_id', 'overall_rating']]
    temp = temp.groupby('player_api_id').mean()
    return temp['overall_rating'].sum()

In [25]:
players_per_team_df['total_score'] = players_per_team_df.apply(calculate_total_score, axis=1)

players_per_team_df['average_score'] = players_per_team_df.total_score/players_per_team_df.players.apply(len)
players_per_team_df.sample(5)

,season,team_api_id,players,total_score,average_score,team_long_name
126,2012/2013,8528,"[26631, 35472, 118929, 49825, 71844, 212389, 1...",1647.000000,71.608696,Wigan Athletic
750,2014/2015,9865,"[99333, 45204, 434325, 37656, 150296, 113311, ...",1508.895238,68.586147,UD Almería
221,2015/2016,9941,"[303360, 166020, 210184, 169865, 435214, 19074...",1567.083333,68.134058,Toulouse FC
568,2010/2011,9880,"[39306, 73999, 39701, 41243, 155297, 41380, 39...",1688.500000,70.354167,Cesena
459,2015/2016,8535,"[141699, 47373, 38433, 414245, 113318, 38699, ...",1736.482179,75.499225,Fiorentina


In [26]:
players_per_team_df = players_per_team_df.merge(team[['team_api_id', 'team_long_name']], left_on='team_api_id', right_on='team_api_id')
players_per_team_df.head()

,season,team_api_id,players,total_score,average_score,team_long_name_x,team_long_name_y
0,2008/2009,10260,"[34944, 40961, 30726, 67850, 30865, 30615, 303...",2094.5,74.803571,Manchester United,Manchester United
1,2009/2010,10260,"[34944, 40961, 30726, 67850, 30865, 30615, 303...",1908.0,76.320000,Manchester United,Manchester United
2,2010/2011,10260,"[34944, 40961, 30726, 70409, 30865, 30615, 303...",2077.0,76.925926,Manchester United,Manchester United
3,2011/2012,10260,"[34944, 40961, 182917, 70409, 30865, 186137, 3...",1770.5,73.770833,Manchester United,Manchester United
4,2012/2013,10260,"[34944, 40961, 182917, 70409, 30865, 186137, 3...",1930.5,80.437500,Manchester United,Manchester United


In [27]:
match['result'] = np.sign(match.home_team_goal - match.away_team_goal)
match.sample(5)

,id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,goal,card,home_players,away_players,result
22995,22996,21518,2011/2012,5,2011-09-21 00:00:00,1051697,8370,8581,1,2,<goal><value><comment>n</comment><stats><goals...,<card><value><comment>y</comment><stats><ycard...,"[196126.0, 183449.0, 33961.0, 172147.0, 33898....","[33753.0, 37597.0, 33847.0, 37672.0, 37484.0, ...",-1
10130,10131,7809,2015/2016,28,2016-04-02 00:00:00,2002336,9823,9810,1,0,<goal><value><comment>n</comment><stats><goals...,<card><value><comment>y</comment><stats><ycard...,"[27299.0, 30894.0, 37410.0, 121633.0, 282691.0...","[66757.0, 158234.0, 106309.0, 18904.0, 155534....",1
11872,11873,10257,2012/2013,2,2012-09-01 00:00:00,1286985,9804,9878,3,0,<goal><value><comment>npm</comment><event_inci...,<card><value><comment>y</comment><stats><ycard...,"[41818.0, 46820.0, 150670.0, 49543.0, 39391.0,...","[181910.0, 190971.0, 39541.0, 215723.0, 39485....",1
4444,4445,1729,2015/2016,14,2015-11-29 00:00:00,1988846,9850,9825,1,1,<goal><value><comment>n</comment><stats><goals...,<card><value><comment>y</comment><stats><ycard...,"[40602.0, 265918.0, 72417.0, 26209.0, 25667.0,...","[30859.0, 427438.0, 35606.0, 46539.0, 38521.0,...",0
6665,6666,4769,2012/2013,9,2012-10-20 00:00:00,1216898,9851,10249,2,1,<goal><value><comment>n</comment><stats><goals...,<card><value><comment>y</comment><stats><ycard...,"[11319.0, 159882.0, 41120.0, 103470.0, 179517....","[33109.0, 26325.0, 26399.0, 26138.0, 41360.0, ...",1


In [30]:
players_per_team_df['total_points'] = 0
for season, df in match.groupby('season'):
    mask = players_per_team_df.season == season
    for idx, row in df.iterrows():
        if row.result == 1:
            players_per_team_df.loc[np.logical_and(mask, players_per_team_df.team_api_id == row.home_team_api_id), 'total_points'] += 3
        elif row.result == -1:
            players_per_team_df.loc[np.logical_and(mask, players_per_team_df.team_api_id == row.away_team_api_id), 'total_points'] += 3
        else:
            players_per_team_df.loc[np.logical_and(mask, players_per_team_df.team_api_id == row.home_team_api_id), 'total_points'] += 1
            players_per_team_df.loc[np.logical_and(mask, players_per_team_df.team_api_id == row.away_team_api_id), 'total_points'] += 1


players_per_team_df.sample(5)

,season,team_api_id,players,total_score,average_score,team_long_name_x,team_long_name_y,total_points
659,2010/2011,9783,"[96651, 167069, 37793, 18853, 45744, 33844, 34...",1970.500000,70.375000,RC Deportivo de La Coruña,RC Deportivo de La Coruña,43
376,2008/2009,9904,"[37380, 39045, 30854, 27654, 25483, 23819, 272...",1751.000000,72.958333,Hannover 96,Hannover 96,40
459,2015/2016,8535,"[141699, 47373, 38433, 414245, 113318, 38699, ...",1736.482179,75.499225,Fiorentina,Fiorentina,64
386,2010/2011,8226,"[37381, 147729, 27411, 242709, 39841, 157729, ...",1614.500000,70.195652,TSG 1899 Hoffenheim,TSG 1899 Hoffenheim,43
20,2012/2013,8472,"[30352, 46353, 23073, 24230, 36786, 111801, 33...",1488.500000,70.880952,Sunderland,Sunderland,39


In [33]:
#players_per_team_df.drop('team_long_name_x', axis=1, inplace=True)
players_per_team_df.rename(columns={'team_long_name_y': 'team_long_name'}, inplace=True)
players_per_team_df.sample(5)

,season,team_api_id,players,total_score,average_score,team_long_name,total_points
58,2010/2011,8455,"[30853, 163205, 30856, 30859, 31906, 30627, 52...",1565.500000,82.394737,Chelsea,71
671,2011/2012,9869,"[74752, 75395, 195335, 202639, 34582, 107929, ...",1834.000000,65.500000,Real Sporting de Gijón,37
172,2012/2013,8521,"[167557, 26251, 21776, 421785, 39965, 93471, 2...",1907.500000,65.775862,Stade Brestois 29,29
530,2015/2016,9857,"[447488, 362115, 518663, 415113, 154249, 30068...",1975.925214,70.568758,Bologna,42
295,2015/2016,8178,"[215168, 37373, 103686, 70409, 667787, 261653,...",1782.907937,74.287831,Bayer 04 Leverkusen,60


In [34]:
players_per_team_df.to_csv('final_df.csv',index_label='id')